In [8]:
import googlemaps
import pandas as pd
import numpy as np
import csv
from src.utils import get_api_key, now
from pprint import pprint
import json
import pickle

In [9]:
with open("./output/mpc_pairs.csv") as fp:
    mpc_pairs = list(csv.DictReader(fp))

pprint(mpc_pairs[:5])

[{'latitude_1': '1.42513621',
  'latitude_2': '1.456123',
  'longitude_1': '103.61443042',
  'longitude_2': '103.761701',
  'master_postcode_1': '79000',
  'master_postcode_2': '80000',
  'ppv_region': 'Johor'},
 {'latitude_1': '1.42513621',
  'latitude_2': '1.662964',
  'longitude_1': '103.61443042',
  'longitude_2': '103.600178',
  'master_postcode_1': '79000',
  'master_postcode_2': '81000',
  'ppv_region': 'Johor'},
 {'latitude_1': '1.42513621',
  'latitude_2': '1.606506',
  'longitude_1': '103.61443042',
  'longitude_2': '103.647617',
  'master_postcode_1': '79000',
  'master_postcode_2': '81400',
  'ppv_region': 'Johor'},
 {'latitude_1': '1.42513621',
  'latitude_2': '1.876001',
  'longitude_1': '103.61443042',
  'longitude_2': '103.614046',
  'master_postcode_1': '79000',
  'master_postcode_2': '81440',
  'ppv_region': 'Johor'},
 {'latitude_1': '1.42513621',
  'latitude_2': '1.75050278',
  'longitude_1': '103.61443042',
  'longitude_2': '103.672815',
  'master_postcode_1': '7900

In [10]:
# Let's separate this by ppv_region:
mpc_pairs_by_region = dict()
for mpc_pair in mpc_pairs:
    ppv = mpc_pair["ppv_region"]
    if ppv not in mpc_pairs_by_region.keys():
        mpc_pairs_by_region[ppv] = []
    mpc_pairs_by_region[ppv].append(mpc_pair)

# Sample: Kedah
pprint(mpc_pairs_by_region["Kedah"][:5])

# Number of pairs by state:
pprint({k: len(v) for k, v in mpc_pairs_by_region.items()})


[{'latitude_1': '6.131583',
  'latitude_2': '6.269412',
  'longitude_1': '100.373412',
  'longitude_2': '100.42052',
  'master_postcode_1': '05000',
  'master_postcode_2': '06000',
  'ppv_region': 'Kedah'},
 {'latitude_1': '6.131583',
  'latitude_2': '6.437103',
  'longitude_1': '100.373412',
  'longitude_2': '100.428782',
  'master_postcode_1': '05000',
  'master_postcode_2': '06010',
  'ppv_region': 'Kedah'},
 {'latitude_1': '6.131583',
  'latitude_2': '6.51576667',
  'longitude_1': '100.373412',
  'longitude_2': '100.421848',
  'master_postcode_1': '05000',
  'master_postcode_2': '06050',
  'ppv_region': 'Kedah'},
 {'latitude_1': '6.131583',
  'latitude_2': '6.392752',
  'longitude_1': '100.373412',
  'longitude_2': '100.305771',
  'master_postcode_1': '05000',
  'master_postcode_2': '06100',
  'ppv_region': 'Kedah'},
 {'latitude_1': '6.131583',
  'latitude_2': '5.610519',
  'longitude_1': '100.373412',
  'longitude_2': '100.444822',
  'master_postcode_1': '05000',
  'master_postcod

In [12]:
mpc_pairs_by_region.keys()

dict_keys(['Johor', 'Kedah', 'Kelantan', 'Melaka', 'Negeri Sembilan', 'Pahang', 'Perak', 'Perlis', 'Pulau Pinang', 'Sabah', 'Sarawak', 'Selangor/KL/Putrajaya', 'Terengganu'])

In [33]:
mpc_pairs_api = mpc_pairs_by_region["Sarawak"]

SKIP = True # Change this to False (and comment out the sample above) to use the Maps API
if not SKIP:
    gmaps = googlemaps.Client(key=get_api_key())
    responses = []
    ts = now()
    with open(f"./responses/distance-matrix-{ts}.json", mode="a") as fp:
        for item in mpc_pairs_api:
            # coordinates separated by comma, no spaces
            origin = f"{item['latitude_1']},{item['longitude_1']}"
            destination = f"{item['latitude_2']},{item['longitude_2']}"
            try:
                response = gmaps.distance_matrix(origin, destination)
                # print(response)
            except Exception as exc:
                print(exc)
            responses.append(response)
        json.dump(responses, fp)

In [37]:
# with open(f"./responses/distance-matrix-{ts}.json") as fp: # Use this line for MAPS API
with open("./responses/distance-matrix-20210606-221535-sarawak.json") as fp: # Use this line for sample
    responses = json.load(fp)

assert len(mpc_pairs_api) == len(responses)

# Add the corresponding duration into the mpc_pairs dictionaries
try:
    for item, resp in zip(mpc_pairs_api, responses):
        element = resp["rows"][0]["elements"][0]
        if element["status"] == "ZERO_RESULTS":
            item["duration"] = "inf"
        else:
            item["duration"] = element["duration"]["value"]
except KeyError as exc:
    print(resp)
    raise exc

In [38]:
# time matrix
# lets reuse the distance matrix

# Using pickle for now, because importing a csv/Excel causes certain postcodes to become integers instead of strings.
# The pickle file is generated at the end of Notebook #1.
with open("./output/distance_matrix.pickle", mode="rb") as fp:
    df_mat = pickle.load(fp)
df_mat = df_mat.applymap(lambda x: 0, na_action="ignore") # Change all numbers to 0.


In [39]:
# [[[0, 605], [535, 0]], [[0, 2558], [2597, 0]]]
for item in mpc_pairs_api:
    mpc1 = (item["ppv_region"], item["master_postcode_1"])
    mpc2 = (item["ppv_region"], item["master_postcode_2"])
    duration = item["duration"] if item["duration"] != "inf" else np.inf
    df_mat.loc[mpc1, mpc2] = duration
    df_mat.loc[mpc2, mpc1] = duration # also add across the diagonal



In [40]:
result = df_mat[df_mat>0].dropna(how="all", axis=0).dropna(how="all", axis=1)  # View the data
for i in range(len(result)):
    result.iloc[i, i] = 0 # Set diagonals to zero
result

Sarawak                                                          \
                09130  93000  94000  94200  94300  94500  94600  94650  94700   
Sarawak 09130       0  30675  31979  29448  29069  36103  27302  16686  28048   
        93000   30675      0   2853   2549   1952   6528   3559  18306   4095   
        94000   31979   2853      0   2996   3412   4407   5019  19766   4542   
        94200   29448   2549   2996      0   1495   7278   3101  17849   1706   
        94300   29069   1952   3412   1495      0   7412   1971  16718   2912   
        94500   36103   6528   4407   7278   7412      0   9055  23803   8737   
        94600   27302   3559   5019   3101   1971   9055      0  14966   4033   
        94650   16686  18306  19766  17849  16718  23803  14966      0  16514   
        94700   28048   4095   4542   1706   2912   8737   4033  16514      0   
        94800   27104   6214   7674   5756   4626  11711   2874  14846   2881   
        94850   23912   7121   8580   6663   5533  12617   3780  11654   7680   
        94900   24769  14585  15032  12196  13401  19227  14130  13235  10567   
        94950   17480  13212  14672  12754  11624  18708   9871   5223  13506   
        95000   21788  11604  12050   9215  10420  16246  11149  10254   7586   
        95300   14726  16347  17807  15889  14759  21844  13006   1909  14665   
        95400   15378  15989  17449  15532  14401  21486  12649   3120  14308   
        95500   17073  14784  16243  14181  13196  20280  11443   4815  12553   
        95600   17590  15857  16409  13573  14269  20605  12516   5888  11945   
        95700   17937  15125  15572  12736  13942  19767  13031   6403  11107   
        95800   19884  12710  13157  10321  11527  17352  12255   8351   8692   
        95900   21028  15213  15660  12824  14030  19856  14759   9495  11196   
        96000    9262  22017  23477  21360  20429  27514  18677   8134  19732   
        96100   11971  18964  20424  18507  17376  24461  15624   5081  17283   
        96150   12841  26347  27807  25691  24759  31844  23007  12465  24062   
        96200     inf    inf    inf    inf    inf    inf    inf    inf    inf   
        96250     inf    inf    inf    inf    inf    inf    inf    inf    inf   
        96300    5198  32563  34023  31906  30975  38060  29223  18680  30278   
        96400    2991  30356  31816  29699  28768  35852  27015  16473  28071   
        96500   11171  19836  21296  19379  18248  25333  16496   5954  18155   
        96600    9956  21171  22631  20319  19583  26668  17831   7289  18691   
        96700    9883  22673  24133  21821  21085  28170  19333   8790  20193   
        96800   16210  28652  30112  27800  27064  34149  25311  14769  26171   
        96850   13948  26389  27849  25537  24801  31886  23049  12507  23909   
        96900   17296  47785  49245  47129  46197  53282  44445  33903  45500   
        97000    5494  35984  37444  35327  34396  41481  32644  22101  33699   
        97100    7481  37971  39430  37314  36383  43467  34630  24088  35685   
        97200    5005  32406  33866  31750  30818  37903  29066  18524  30121   
        98000   15292  45782  47242  45125  44194  51278  42441  31899  43497   
        98050   21942  52432  53891  51775  50844  57928  49091  38549  50146   
        98100   15621  46110  47570  45454  44522  51607  42770  32228  43825   
        98150   12608  43098  44558  42441  41510  48595  39757  29215  40813   
        98200   11359  41849  43308  41192  40260  47345  38508  27966  39563   
        98300   17332  47821  49281  47165  46233  53318  44481  33938  45536   
        98700   25373  55863  57322  55206  54275  61359  52522  41980  53577   
        98800   40038  70528  71987  69871  68940  76024  67187  56645  68243   

                      ...                                                   \
               94800  ...  97100  97200  98000  98050  98100  98150  98200   
Sarawak 09130  27104  ...   7481   5005  15292  21

In [41]:
result.index = result.index.droplevel()
result.columns = result.columns.droplevel()


In [42]:
result

,09130,93000,94000,94200,94300,94500,94600,94650,94700,94800,...,97100,97200,98000,98050,98100,98150,98200,98300,98700,98800
09130,0,30675,31979,29448,29069,36103,27302,16686,28048,27104,...,7481,5005,15292,21942,15621,12608,11359,17332,25373,40038
93000,30675,0,2853,2549,1952,6528,3559,18306,4095,6214,...,37971,32406,45782,52432,46110,43098,41849,47821,55863,70528
94000,31979,2853,0,2996,3412,4407,5019,19766,4542,7674,...,39430,33866,47242,53891,47570,44558,43308,49281,57322,71987
94200,29448,2549,2996,0,1495,7278,3101,17849,1706,5756,...,37314,31750,45125,51775,45454,42441,41192,47165,55206,69871
94300,29069,1952,3412,1495,0,7412,1971,16718,2912,4626,...,36383,30818,44194,50844,44522,41510,40260,46233,54275,68940
94500,36103,6528,4407,7278,7412,0,9055,23803,8737,11711,...,43467,37903,51278,57928,51607,48595,47345,53318,61359,76024
94600,27302,3559,5019,3101,1971,9055,0,14966,4033,2874,...,34630,29066,42441,49091,42770,39757,38508,44481,52522,67187
94650,16686,18306,19766,17849,16718,23803,14966,0,16514,14846,...,24088,18524,31899,38549,32228,29215,27966,33938,41980,56645
94700,28048,4095,4542,1706,2912,8737,4033,16514,0,2881,...,35685,30121,43497,50146,43825,40813,39563,45536,53577,68243
94800,27104,6214,7674,5756,4626,11711,2874,14846,2881,0,...,34487,28923,42298,48948,42627,39614,38365,44338,52379,67044


In [43]:
name = "sarawak"
result_min = result // 60
result.to_csv(f"./output/time-matrix-{name}-sec.csv")
result_min.to_csv(f"./output/time-matrix-{name}-minfloored.csv")
